# Pre-processing and Training Data Development

In [1]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import category_encoders as ce
from category_encoders import TargetEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest, f_regression

from library.sb_utils import save_file

In [2]:
path = r"C:\Users\adame\OneDrive\Documents\GitHub\springboard_repository\Capstone_books\Capstone_books\data\processed\cleaned_books.csv"
df = pd.read_csv(path)

In [3]:
print(df.shape)
df.head()

(11123, 15)


,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,average_rating_mean,ratings_count_sum,text_reviews_count_sum
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,2006-09-16,Scholastic Inc.,4.57,2096903,27669
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,2004-09-01,Scholastic Inc.,4.49,2153167,29221
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,2003-11-01,Scholastic,4.42,2300296,34936
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,2004-05-01,Scholastic Inc.,4.56,2342726,36465
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,2004-09-13,Scholastic,4.78,41428,164


In [4]:
# There are two missing dates
df[df.isnull().any(axis=1)]

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,average_rating_mean,ratings_count_sum,text_reviews_count_sum
8177,31373,In Pursuit of the Proper Sinner (Inspector Lyn...,Elizabeth George,4.10,0553575104,9780553575101,eng,718,10608,295,NaN,Bantam Books,4.10,10608,295
11094,45531,Montaillou village occitan de 1294 à 1324,Emmanuel Le Roy Ladurie/Emmanuel Le Roy-Ladurie,3.96,2070323285,9782070323289,fre,640,15,2,NaN,Folio histoire,3.96,15,2


In [5]:
# the column `publication_date` has two `NaN` values, inserting `publication_date`
df.at[8177, 'publication_date'] = '1999-01-01'
df.at[11094, 'publication_date'] = '1975-01-01'

In [6]:
# group/combine books by title
combined_df = df.drop_duplicates(subset=['title'])

In [7]:
combined_df.shape

(10348, 15)

In [8]:
combined_df.head()

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,average_rating_mean,ratings_count_sum,text_reviews_count_sum
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,2006-09-16,Scholastic Inc.,4.57,2096903,27669
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,2004-09-01,Scholastic Inc.,4.49,2153167,29221
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,2003-11-01,Scholastic,4.42,2300296,34936
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,2004-05-01,Scholastic Inc.,4.56,2342726,36465
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,2004-09-13,Scholastic,4.78,41428,164


In [9]:
# modify dataframe, remove redundant columns `average_rating`, `ratings_count`, `text_reviews_count`.
combined_df.drop(columns=['average_rating', 'ratings_count', 'text_reviews_count'], inplace=True)

C:\Users\adame\AppData\Local\Temp\ipykernel_6504\1236559507.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df.drop(columns=['average_rating', 'ratings_count', 'text_reviews_count'], inplace=True)


In [10]:
combined_df.reset_index(drop=True, inplace=True)

In [11]:
combined_df.head()

,bookID,title,authors,isbn,isbn13,language_code,num_pages,publication_date,publisher,average_rating_mean,ratings_count_sum,text_reviews_count_sum
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,0439785960,9780439785969,eng,652,2006-09-16,Scholastic Inc.,4.57,2096903,27669
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,0439358078,9780439358071,eng,870,2004-09-01,Scholastic Inc.,4.49,2153167,29221
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,0439554896,9780439554893,eng,352,2003-11-01,Scholastic,4.42,2300296,34936
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,043965548X,9780439655484,eng,435,2004-05-01,Scholastic Inc.,4.56,2342726,36465
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,0439682584,9780439682589,eng,2690,2004-09-13,Scholastic,4.78,41428,164


### Add new features

In [12]:
# some books have zero `text_review_count`, so I added a 1 to those rows
combined_df.loc[combined_df['text_reviews_count_sum'] == 0, 'text_reviews_count_sum'] += 1

In [13]:
combined_df['publication_year'] = pd.to_datetime(combined_df['publication_date']).dt.year
combined_df['pages_per_review'] = combined_df['num_pages'] / combined_df['text_reviews_count_sum']
combined_df['publisher_count'] = combined_df.groupby('publisher')['publisher'].transform('count')

C:\Users\adame\AppData\Local\Temp\ipykernel_6504\809790304.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df['publication_year'] = pd.to_datetime(combined_df['publication_date']).dt.year
C:\Users\adame\AppData\Local\Temp\ipykernel_6504\809790304.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df['pages_per_review'] = combined_df['num_pages'] / combined_df['text_reviews_count_sum']
C:\Users\adame\AppData\Local\Temp\ipykernel_6504\809790304.py:3: SettingWithCopyWarning: 
A value is 

In [14]:
combined_df.head()

,bookID,title,authors,isbn,isbn13,language_code,num_pages,publication_date,publisher,average_rating_mean,ratings_count_sum,text_reviews_count_sum,publication_year,pages_per_review,publisher_count
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,0439785960,9780439785969,eng,652,2006-09-16,Scholastic Inc.,4.57,2096903,27669,2006,0.023564,12
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,0439358078,9780439358071,eng,870,2004-09-01,Scholastic Inc.,4.49,2153167,29221,2004,0.029773,12
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,0439554896,9780439554893,eng,352,2003-11-01,Scholastic,4.42,2300296,34936,2003,0.010076,31
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,043965548X,9780439655484,eng,435,2004-05-01,Scholastic Inc.,4.56,2342726,36465,2004,0.011929,12
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,0439682584,9780439682589,eng,2690,2004-09-13,Scholastic,4.78,41428,164,2004,16.402439,31


### Encode categorical columns

In [15]:
le = LabelEncoder()
enc = OneHotEncoder(sparse=False, sparse_output=False, drop='first')
cat_enc = TargetEncoder()

In [16]:
# encode book title column
combined_df['title_encoded'] = le.fit_transform(combined_df['title'])

C:\Users\adame\AppData\Local\Temp\ipykernel_6504\2598974073.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df['title_encoded'] = le.fit_transform(combined_df['title'])


In [17]:
# encode author column
combined_df['authors_encoded'] = le.fit_transform(combined_df['authors'])

C:\Users\adame\AppData\Local\Temp\ipykernel_6504\2515486910.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df['authors_encoded'] = le.fit_transform(combined_df['authors'])


In [18]:
# one hot encode `language_code`
language_code_encoded = enc.fit_transform(combined_df[['language_code']])

language_code_encoded_df = pd.DataFrame(language_code_encoded, columns=enc.get_feature_names_out(['language_code']))
combined_df = pd.concat([combined_df, language_code_encoded_df], axis=1)

C:\Users\adame\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [20]:
# Target encode `publisher` column
combined_df['publisher_encoded'] = cat_enc.fit_transform(combined_df['publisher'], combined_df['average_rating_mean'])

In [54]:
combined_df.shape

(10348, 43)

### Split data into train and test datasets

In [37]:
X = combined_df.drop(columns=['title', 'authors', 'language_code', 'bookID', 'isbn13', 'isbn', 'publication_date', 'publisher', 'average_rating_mean'], axis=1)
y = combined_df.average_rating_mean

In [38]:
X.shape

(10348, 34)

In [39]:
y.shape

(10348,)

In [40]:
# split dataframe into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

### Scale the data

In [51]:
# create a scaler object, fit scaler to data and then transform the data
scaler = StandardScaler()
scaler.fit(X_train)
X_scaled = scaler.transform(X_train)

# Modeling

#### Model 1 Linear Regression

In [52]:
model1 = LinearRegression()
model1.fit(X_scaled, y_train)

LinearRegression()

In [56]:
y_test_pred = model1.predict(X_test)

C:\Users\adame\anaconda3\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


In [57]:
r2_score(y_test, y_test_pred)

-6.797898777497372e+23

#### Model 2 

In [45]:
model2 = RandomForestRegressor(n_estimators=100, random_state=42)
model2.fit(X_train_scaled, y_train)

RandomForestRegressor(random_state=42)

In [58]:
y_test_pred_model2 = model2.predict(X_test)

C:\Users\adame\anaconda3\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


In [59]:
r2_score(y_test, y_test_pred_model2)

-1.0027426667315091

In [34]:
data_path = '../data/processed'
save_file(combined_df, 'cleaned_processed_books.csv', data_path)

A file already exists with this name.

Do you want to overwrite? (Y/N)y
Writing file.  "../data/processed\cleaned_processed_books.csv"
